In [2]:
import pandas as pd
from collections import Counter

filmes = pd.read_csv("movies.csv")
filmes.columns = ["Filme ID", "Título", "Gêneros"]
filmes = filmes.set_index("Filme ID")
filmes.head()

,Título,Gêneros
Filme ID,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [3]:
notas = pd.read_csv("ratings.csv")
notas.columns = ["Usuário ID", "Filme ID", "Nota", "Data"]
notas.head()

,Usuário ID,Filme ID,Nota,Data
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
filmes.describe()

,Título,Gêneros
count,9742,9742
unique,9737,951
top,Emma (1996),Drama
freq,2,1053


In [5]:
notas.describe()

,Usuário ID,Filme ID,Nota,Data
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


# Primeira tentativa de recomendação

## Heurística de Total de Votos

In [18]:
users_interests = [
        
["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"], 
["Python", "scikit-learn", "scipy", "numpy", "statsmodel", "pandas"], 
["R", "Python", "statistics", "regression", "probability"], 
["machine learning", "regression", "decission trees", "libsvm"], 
["Python", "R", "Java", "C++", "Haskell", "programming languages"],
["statistics", "probability", "mathematics", "theory"],
["pandas", "R", "Python"]
            
]

popular_interests = Counter(interest for users_interests in users_interests for interest in users_interests).most_common()

print(popular_interests)

[('Python', 4), ('R', 3), ('HBase', 2), ('Java', 2), ('Cassandra', 2), ('pandas', 2), ('statistics', 2), ('regression', 2), ('probability', 2), ('Hadoop', 1), ('Big Data', 1), ('Spark', 1), ('Storm', 1), ('NoSQL', 1), ('MongoDB', 1), ('Postgres', 1), ('scikit-learn', 1), ('scipy', 1), ('numpy', 1), ('statsmodel', 1), ('machine learning', 1), ('decission trees', 1), ('libsvm', 1), ('C++', 1), ('Haskell', 1), ('programming languages', 1), ('mathematics', 1), ('theory', 1)]


In [19]:
total_de_votos = notas["Filme ID"].value_counts()
total_de_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: Filme ID, dtype: int64

In [20]:
filmes.loc[318]

Título            Shawshank Redemption, The (1994)
Gêneros                                Crime|Drama
total_de_votos                                 317
nota_media                                 4.42902
Name: 318, dtype: object

Ordenar pelo mais votado e recomendar os primeiros

In [21]:
filmes['total_de_votos'] = total_de_votos
filmes.head()

,Título,Gêneros,total_de_votos,nota_media
Filme ID,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818
3,Grumpier Old Men (1995),Comedy|Romance,52.0,3.259615
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0,2.357143
5,Father of the Bride Part II (1995),Comedy,49.0,3.071429


In [22]:
filmes.sort_values("total_de_votos", ascending = False).head(10)

,Título,Gêneros,total_de_votos,nota_media
Filme ID,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


In [23]:
notas_medias = notas.groupby("Filme ID").mean()["Nota"]
notas_medias.head()

Filme ID
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: Nota, dtype: float64

In [12]:
filmes["nota_media"] = notas_medias
filmes.sort_values("total_de_votos", ascending = False).head(10)

,Título,Gêneros,total_de_votos,nota_media
Filme ID,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


# Outra Heurística

## Nota média e Filtrando Votos

In [24]:
filmes.sort_values("nota_media", ascending = False).head(10)

,Título,Gêneros,total_de_votos,nota_media
Filme ID,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


In [25]:
filmes.query("total_de_votos >= 10").sort_values("nota_media", ascending = False).head(10)

,Título,Gêneros,total_de_votos,nota_media
Filme ID,,,,
1041,Secrets & Lies (1996),Drama,11.0,4.590909
3451,Guess Who's Coming to Dinner (1967),Drama,11.0,4.545455
1178,Paths of Glory (1957),Drama|War,12.0,4.541667
1104,"Streetcar Named Desire, A (1951)",Drama,20.0,4.475000
2360,"Celebration, The (Festen) (1998)",Drama,12.0,4.458333
1217,Ran (1985),Drama|War,15.0,4.433333
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
951,His Girl Friday (1940),Comedy|Romance,14.0,4.392857
1927,All Quiet on the Western Front (1930),Action|Drama|War,10.0,4.350000


In [44]:
filmes_com_mais_de_50_votos = filmes.query("total_de_votos >= 50").sort_values("nota_media", ascending = False).head(10)
filmes_com_mais_de_50_votos

,Título,Gêneros,total_de_votos,nota_media
Filme ID,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


In [45]:
eu_assisti = [1, 21, 19, 10, 11, 7, 2]
filmes.loc[eu_assisti]

,Título,Gêneros,total_de_votos,nota_media
Filme ID,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
21,Get Shorty (1995),Comedy|Crime|Thriller,89.0,3.494382
19,Ace Ventura: When Nature Calls (1995),Comedy,88.0,2.727273
10,GoldenEye (1995),Action|Adventure|Thriller,132.0,3.496212
11,"American President, The (1995)",Comedy|Drama|Romance,70.0,3.671429
7,Sabrina (1995),Comedy|Romance,54.0,3.185185
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818


In [51]:
x = filmes.query("total_de_votos >= 50").sort_values("nota_media", ascending = False).head(10)
x

,Título,Gêneros,total_de_votos,nota_media
Filme ID,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


In [61]:
notas = pd.read_csv("ratings.csv")
notas.columns = ["Usuário ID", "Filme ID", "Nota", "Data"]
notas["Nota"]

0         4.0
1         4.0
2         4.0
3         5.0
4         5.0
5         3.0
6         5.0
7         4.0
8         5.0
9         5.0
10        5.0
11        5.0
12        3.0
13        5.0
14        4.0
15        5.0
16        3.0
17        3.0
18        5.0
19        4.0
20        4.0
21        5.0
22        4.0
23        3.0
24        4.0
25        5.0
26        4.0
27        3.0
28        5.0
29        4.0
         ... 
100806    3.0
100807    4.0
100808    4.0
100809    3.5
100810    3.5
100811    5.0
100812    4.5
100813    4.0
100814    5.0
100815    3.5
100816    3.5
100817    3.0
100818    3.0
100819    3.0
100820    2.5
100821    4.5
100822    3.0
100823    3.0
100824    4.0
100825    4.0
100826    3.5
100827    3.5
100828    3.5
100829    5.0
100830    4.0
100831    4.0
100832    5.0
100833    5.0
100834    5.0
100835    3.0
Name: Nota, Length: 100836, dtype: float64

In [62]:
type(notas["Nota"])

pandas.core.series.Series

In [64]:
lista = notas['Nota'].tolist()

In [65]:
lista

[4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 3.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 4.0,
 5.0,
 3.0,
 3.0,
 5.0,
 4.0,
 4.0,
 5.0,
 4.0,
 3.0,
 4.0,
 5.0,
 4.0,
 3.0,
 5.0,
 4.0,
 4.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 3.0,
 5.0,
 3.0,
 4.0,
 3.0,
 3.0,
 4.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 3.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 3.0,
 3.0,
 3.0,
 3.0,
 4.0,
 4.0,
 5.0,
 4.0,
 5.0,
 3.0,
 5.0,
 5.0,
 4.0,
 5.0,
 3.0,
 3.0,
 5.0,
 4.0,
 4.0,
 5.0,
 4.0,
 4.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 4.0,
 2.0,
 4.0,
 4.0,
 5.0,
 5.0,
 2.0,
 5.0,
 4.0,
 5.0,
 2.0,
 5.0,
 4.0,
 3.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 3.0,
 5.0,
 5.0,
 5.0

In [66]:
type(lista)

list